In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

import glob

In [9]:
pdfs = glob.glob("../pdfs/*.pdf")

documents = []
for pdf in pdfs:
    _loader = PyPDFLoader(pdf).load()
    documents += _loader
    print(f"{len(_loader)}/{len(documents)}")
    

248/248
47/295
129/424


In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex = False,
)

docs = text_splitter.split_documents(documents)

In [10]:
embeddings = HuggingFaceEmbeddings(
    model_name = "firqaaa/indo-sentence-bert-base",
    cache_folder = "./cache-indo-sentence/"
)

In [13]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    path="./local_qdrant_multi",
    collection_name="dokumen-filkom",
)

In [22]:
import getpass
import os

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

### Simple Retriever

In [111]:
from langchain.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = OpenAI(
    model_name="gpt-3.5-turbo",
)

QA_PROMPT_1 = PromptTemplate(
    input_variables=["query", "contexts"],
    template="""Anda adalah chatbot Universitas Brawijaya yang tugasnya adalah membantu \
        pertanyaan dari tamu. Apabila suatu pertanyaan tidak dapat dijawab dari \
        informasi yang diberikan, maka jawab dengan "Saya Tidak Bisa Menjawab Pertanyaan Di Luar Universitas Brawijaya".

    Konteks:
    {contexts}

    Pertanyaan: \n{query}
    Jawaban: \n""",
)

qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT_1)

def ask_1 (query):
    docs = qdrant.similarity_search(query, k=5)
    answers = qa_chain.run({
        "query": query,
        "contexts": "\n---\n".join([d.page_content for d in docs])
    })
    
    return answers, docs

/data/home/abiyyu/miniconda3/envs/biyu/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/data/home/abiyyu/miniconda3/envs/biyu/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [132]:
qs = [
    "Bagaimana standar kriteria penilaian seminar hasil?",
    "Berapakah jumlah SKS yang harus di selesaikan seorang mahasiswa agar dikatakan lulus?",
    "Apa itu transfer kredit dan pengakuan kredit?",
    "Apa yang dimaksud dengan string theory?",
    "Jelaskan peraturan tentang kecurangan dan sanksi!",
    "Apa saja referensi yang dapat digunakan untuk mata kuliah Rekayasa Perangkat Lunak?",
]

for i, q in enumerate(qs):
    answer = ask_1(q)[0]
    print(f"A{i + 1}: {answer}")

A1: Kriteria penilaian seminar hasil adalah sebagai berikut:
1. Sistematika Presentasi (A): menilai kelengkapan presentasi yang disusun mahasiswa. Kelengkapan presentasi meliputi: Latar Belakang, Tujuan, Rumusan Masalah, Tinjauan Pustaka, Landasan teori, Metodologi Penelitian, Pembahasan hasil, Kesimpulan dan Saran.
2. Kemampuan menjelaskan poin-poin penting (B): menilai kemampuan mahasiswa dalam menjelaskan poin-poin penting meliputi: Latar Belakang, Tujuan, Rumusan Masalah, Tinjauan Pustaka, Landasan Teori, Metodologi Penelitian, Pembahasan Hasil, Kesimpulan dan Saran.
3. Teknik presentasi (C): menilai teknik presentasi mahasiswa meliputi: kejelasan suara, artikulasi, intonasi, kepercayaan diri, dan kemampuan mahasiswa dalam menarik perhatian peserta sidang.

Apabila pertanyaan Anda tidak terkait dengan Universitas Brawijaya, maka jawabannya adalah "Saya Tidak Bisa Menjawab Pertanyaan Di Luar Universitas Brawijaya".
A2: Jumlah SKS yang harus diselesaikan oleh seorang mahasiswa agar d

In [156]:
ask_1("Apa saja referensi yang dapat digunakan untuk mata kuliah Rekayasa Perangkat Lunak?")

('Tidak ada informasi yang ditunjukkan dalam konteks yang diberikan tentang referensi yang dapat digunakan untuk mata kuliah Rekayasa Perangkat Lunak.',
 [Document(page_content='akhir yang akan diangkat oleh mahasiswa. Jika dosen pembimbing atau calon dosen pembimbing menganggap bahwa mata kuliah wajib yang sudah ditempuh mencukupi sebagai bekal untuk mengerjakan skripsi/tugas akhir maka mata kuliah pilihan yang diambil dapat berfungsi sebagai pengayaan/penguatan kompetensi. 4. Untuk jalur selain Reguler dan Merdeka Belajar 1 Semester di Dalam UB, topik skripsi/tugas akhir yang akan diangkat disesuaikan dengan kompetensi wajib dasar Prodi dan dapat dikaitkan dengan mata kuliah sesuai paket skema Kurikulum Merdeka yang diambil.  A. Mata Kuliah Pilihan Stream Pengembangan Full Stack  Stream ini memberikan pengayaan kompetensi bagi mahasiswa dalam bidang pengembangan full stack. Beberapa bahan kajian utama yang dibahas pada stream ini antara lain: \x00 Pengelolaan sumber daya proyek peran

### Multi Query

In [159]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

ret = qdrant.as_retriever(
            search_type="mmr",
            search_kwargs={'k': 6, 'lambda_mult': 0.25}
        )
retriever = MultiQueryRetriever.from_llm(
    retriever=ret,
    llm=llm,
)

In [153]:
retriever.get_relevant_documents("Bagaimana aspek kelayakan skripsi?")

[Document(page_content='produk /artefak  utama pada tipe penelitian ini sangat mungkin diperlukan sebuah \nperangkat lunak/keras yang perlu dikembangkan sendiri. Namun demikian, \nperangkat lunak/keras tersebut bukan sebagai fokus kajian dan hanya sebagai \nalat bantu saja untuk menghasil kan kajian yang valid.  \n2.3 Aspek Kelayakan  Skripsi  \nAspek kelayakan skripsi merupakan kriteria mendasar yang harus dipenuhi \noleh sebuah skripsi  untuk menunjukkan bahwa  penelitian skripsi yang akan \ndilakukan oleh mahasiswa sudah sesuai dengan minat studi yang se lama ini \nditekuninya dan mengacu pada pencapaian  salah satu profil lulusan suatu prodi . \nDalam pelaksanaannya, aspek kelayakan ini diukur berdasarkan asesmen \nterhadap usulan skripsi dalam bentuk proposal yang dilakukan di awal proses \nskripsi  oleh Kaprodi . Penilaian tersebut akan dikaji oleh  Kaprodi   sebagai dasar \nuntuk menentukan Calon Pembimbing mahasiswa dalam proses pelaksanaan \nskripsi dan menentukan kesesuaian t

In [154]:

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

QA_PROMPT = PromptTemplate(
    input_variables=["query", "contexts"],
    template="""Anda adalah chatbot Universitas Brawijaya yang tugasnya adalah membantu \
        pertanyaan dari tamu. Apabila suatu pertanyaan tidak dapat dijawab dari \
        informasi yang diberikan, maka jawab dengan "Saya Tidak Bisa Menjawab Pertanyaan Di Luar Universitas Brawijaya".

    Konteks:
    {contexts}

    Pertanyaan: \n{query}
    Jawaban: \n""",
)

qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT)

def ask(questions):
    docs = retriever.get_relevant_documents(questions)
    answers = qa_chain.run({
        "query": questions,
        "contexts": "\n---\n".join([d.page_content for d in docs])
    })
    
    return answers, docs

In [125]:
q1 = ask("Bagaimana standar kriteria penilaian seminar hasil?")
print(q1[0])

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Apa saja kriteria standar yang digunakan untuk menilai hasil seminar?', '2. Bagaimana kriteria penilaian yang biasa dipakai untuk mengevaluasi seminar?', '3. Apa saja faktor yang digunakan untuk menentukan kualitas hasil seminar?']


Standar kriteria penilaian seminar hasil adalah sebagai berikut:

1. Sistematika Presentasi (A): menilai kelengkapan presentasi yang disusun oleh mahasiswa, termasuk latar belakang, tujuan, rumusan masalah, tinjauan pustaka, landasan teori, metodologi penelitian, pembahasan hasil, kesimpulan, dan saran.

2. Kemampuan menjelaskan poin-poin penting (B): menilai kemampuan mahasiswa dalam menjelaskan poin-poin penting dalam presentasi, termasuk latar belakang, tujuan, rumusan masalah, tinjauan pustaka, landasan teori, metodologi penelitian, pembahasan hasil, kesimpulan, dan saran.

3. Teknik presentasi (C): menilai teknik presentasi mahasiswa, termasuk kejelasan suara, artikulasi, intonasi, kepercayaan diri, dan kemampuan mahasiswa dalam menarik perhatian peserta sidang.

4. Sikap dan perilaku (D): menilai sikap dan perilaku mahasiswa selama presentasi, termasuk kerapian pakaian, sikap, dan keantusiasan.

5. Kreativitas presentasi (E): menilai kekreatifan mahasiswa dalam menyusun slide pre

In [126]:
q1[1]

[Document(page_content='yang belum lulus, maka hanya bisa dijadwalkan untuk seminar hasil \n(dalam hal ini, persyaratan menyerahkan transkrip bisa ditangguhkan \ndulu), dan jadwal ujian bisa dilakukan setelah semua nilai lulus \n(persayaratan menyerahkan transkrip sudah dipenuhi).  \n2. Mahasiswa melengkapi  form ulir Persyaratan Seminar Hasil dan Ujian S kripsi \n(SC2 -01) dan form ulir Pendaftaran Seminar Hasil dan U jian Skripsi (SC2 -02). \n3. Akademik J urusan memverifikasi persyaratan.  Apabila persyaratan ini belum \nlengkap , mahasiswa wajib melengkapi nya. \n4. Jika seluruh persyaratan sudah lengkap, maka Sekjur dapat melakukan \npersiapan seminar hasil dan ujian skripsi.  \na. Sekjur menjadwalkan seminar hasil yaitu menentukan jadwal dan \nfasilitator seminar hasil. Jadwal seminar hasil dikeluarkan oleh Akademik \nFakultas dengan jumlah peserta 3 ma hasiswa (untuk 1 sesi seminar). \nFasilitator ditetapk an oleh Sekjur berdasarkan daftar dosen yang \nditetapkan oleh Wakil Deka

In [127]:
print(ask("Berapakah jumlah SKS yang harus di selesaikan seorang mahasiswa agar dikatakan lulus?")[0])

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Apa jumlah SKS yang perlu ditempuh oleh seorang mahasiswa agar memenuhi syarat kelulusan?', '2. Bagaimana jumlah SKS yang harus diselesaikan oleh seorang mahasiswa supaya dapat lulus?', '3. Berapa banyak SKS yang harus diambil oleh seorang mahasiswa agar dianggap lulus?']


Seorang mahasiswa harus menyelesaikan minimal 144 SKS untuk dapat dikatakan lulus.


In [128]:
print(ask("Apa itu transfer kredit dan pengakuan kredit?")[0])

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Bagaimana definisi transfer kredit dan pengakuan kredit?', '2. Apa yang dimaksud dengan transfer kredit dan bagaimana pengakuan kredit bekerja?', '3. Bisa jelaskan apa arti transfer kredit dan pengakuan kredit?']


Transfer kredit adalah proses pengalihan kredit yang diperoleh dari kegiatan pembelajaran di luar program studi ke dalam program studi yang sedang ditempuh. Sedangkan pengakuan kredit adalah proses penghitungan besarnya satuan kredit dan nilai mata kuliah khusus yang diperoleh dari kegiatan merdeka belajar dan dapat dicantumkan di dalam transkrip nilai mahasiswa serta dapat menjadi bagian dari syarat kelulusan program sarjana.


In [129]:
print(ask("Apa yang dimaksud dengan string theory?")[0])

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Bagaimana string theory dapat dijelaskan secara sederhana?', '2. Apa konsep dasar dalam string theory?', '3. Bagaimana string theory berbeda dari teori fisika konvensional?']


Saya Tidak Bisa Menjawab Pertanyaan Di Luar Universitas Brawijaya.


In [130]:
print(ask("Jelaskan peraturan tentang kecurangan dan sanksi!")[0])

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Berikan penjelasan mengenai peraturan yang mengatur tentang tindakan kecurangan dan sanksi apa yang dapat diberikan?', '2. Tolong jelaskan dengan lebih detail mengenai peraturan yang mengatur tentang kecurangan serta jenis-jenis sanksi yang dapat diberikan.', '3. Apa saja peraturan yang ada terkait kecurangan dan apa konsekuensi atau sanksi yang bisa diberikan sesuai peraturan tersebut?']


Peraturan tentang kecurangan dan sanksi terdapat dalam konteks yang Anda berikan. Menurut informasi tersebut, terdapat dua peraturan terkait kecurangan dan sanksi di Universitas Brawijaya. Yang pertama adalah mengenai mahasiswa yang melakukan pelanggaran, jika disertai ancaman kekerasan, pemberian sesuatu, atau tipu muslihat, maka mahasiswa tersebut akan dikenai sanksi berupa dikeluarkan dari Universitas Brawijaya. Yang kedua adalah mahasiswa yang melakukan kecurangan atau plagiarisme dalam pembuatan Tugas Akhir, maka Tugas Akhir dan nilai ujian Tugas Akhirnya akan dibatalkan.


In [166]:
print(ask("Apa saja referensi yang dapat digunakan untuk mata kuliah Rekayasa Perangkat Lunak?")[0])

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Bagaimana cara mendapatkan referensi yang relevan untuk mata kuliah Rekayasa Perangkat Lunak?', '2. Apa saja sumber bacaan yang dapat dijadikan referensi dalam menjalani mata kuliah Rekayasa Perangkat Lunak?', '3. Bagaimana dapat menemukan sumber referensi yang berkualitas untuk studi Rekayasa Perangkat Lunak?']


Anda dapat menggunakan beberapa referensi berikut untuk mata kuliah Rekayasa Perangkat Lunak:
1. Pressman, R.S. (2014). Software Engineering: A Practitioner's Approach. McGraw-Hill Education.
2. Somerville, I. (2015). Software Engineering. Pearson.
3. Ian, S. & Steward, J.M. (2014). Requirements Engineering: Foundation for Software Quality. Springer.
4. Pfleeger, S.L., Atlee, J.M. (2010). Software Engineering: Theory and Practice. Prentice Hall.
5. Mohan, B.K. & Ranjithamalar, M. (2010). Software Engineering. South Asia Books.


In [168]:
print(ask("Berapa SKS mata kuliah Pemrograman Dasar?")[0])

INFO:langchain.retrievers.multi_query:Generated queries: ['Berapa jumlah SKS untuk mata kuliah Pemrograman Dasar?', 'Apa banyaknya SKS yang dibutuhkan untuk mata kuliah Pemrograman Dasar?', 'Berapa SKS yang diperlukan dalam mata kuliah Pemrograman Dasar?']


Mata kuliah Pemrograman Dasar memiliki 3 SKS.


In [172]:
print(ask_1("Berapa SKS mata kuliah Pemrograman Dasar?")[0])

Saya Tidak Bisa Menjawab Pertanyaan Di Luar Universitas Brawijaya.
